In [28]:
# conda environment ml_transplant

import pandas as pd
import pyreadstat
import numpy as np
# from pandas_profiling import ProfileReport
from missingno import matrix
import pickle

In [29]:
data = pd.read_excel("data/ger_data.xlsx")

In [30]:
df_filtered = data.drop(columns=[
    'tx_id',
    'recipient_id',
    'donor_id',
    'tx_center',

    'recipient_disease',
    'los',
    'recipient_cause_of_death',
    'cause_of_graft_failure',
])

In [ ]:
df_new = df_filtered.replace({'recipient_status': 'HU - High Urgent'}, 1)
df_new = df_new.replace({'recipient_status': 'T - Transplantable'}, 2)
df_new = df_new.replace({'recipient_status': 'T2 - Transplantable, acute deterioration'}, 3)
df_new = df_new.replace({'recipient_status': 'T3 - Transplantable, with complications'}, 4)
df_new = df_new.replace({'recipient_status': 'T4 - Transplantable, without complications'}, np.NaN)
df_new = df_new.replace({'recipient_status': 'NT - Not Transplantable'}, np.NaN)

df_new = df_new.replace({'recipient_sex': 'Male'}, 1)
df_new = df_new.replace({'recipient_sex': 'Female'}, 0)

df_new = df_new.replace({'recipient_blood_type': 'A'}, 1)
df_new = df_new.replace({'recipient_blood_type': 'AB'}, 2)
df_new = df_new.replace({'recipient_blood_type': 'B'}, 3)
df_new = df_new.replace({'recipient_blood_type': 'O'}, 4)

df_new = df_new.replace({'recipient_rhesus_factor': 'Negative'}, 0)
df_new = df_new.replace({'recipient_rhesus_factor': 'Positive'}, 1)

df_new = df_new.replace({'donor_sex': 'Female'}, 0)
df_new = df_new.replace({'donor_sex': 'Male'}, 1)

df_new = df_new.replace({'donor_blood_type': 'A'}, 1)
df_new = df_new.replace({'donor_blood_type': 'AB'}, 2)
df_new = df_new.replace({'donor_blood_type': 'B'}, 3)
df_new = df_new.replace({'donor_blood_type': 'O'}, 4)

df_new = df_new.replace({'donor_rhesus_factor': 'Negative'}, 0)
df_new = df_new.replace({'donor_rhesus_factor': 'Positive'}, 1)

df_new = df_new.replace({'donor_region': 'Nordrhein-Westfalen'}, 1)
df_new = df_new.replace({'donor_region': 'Nord'}, 2)
df_new = df_new.replace({'donor_region': 'Bayern'}, 3)
df_new = df_new.replace({'donor_region': 'Mitte'}, 4)
df_new = df_new.replace({'donor_region': 'Ost'}, 5)
df_new = df_new.replace({'donor_region': 'Nord-Ost'}, 6)
df_new = df_new.replace({'donor_region': 'Baden-Württemberg'}, 7)

df_new = df_new.replace({'donor_drugs': 'kein Konsum'}, 0)
df_new = df_new.replace({'donor_drugs': 'oral'}, 1)
df_new = df_new.replace({'donor_drugs': 'ehemals intravenös'}, 2)
df_new = df_new.replace({'donor_drugs': 'intravenös'}, 2)
df_new = df_new.replace({'donor_drugs': 'intravenös in den letzten 3 Monaten'}, 2)
df_new = df_new.replace({'donor_drugs': 'unbekannt'}, np.NaN)

df_new = df_new.replace({'donor_alcohol': 'kein Konsum'}, 0)
df_new = df_new.replace({'donor_alcohol': 'leicht'}, 0) #das musste ich anpassen für die fusion
df_new = df_new.replace({'donor_alcohol': 'mittel'}, 1) #das musste ich anpassen für die fusion
df_new = df_new.replace({'donor_alcohol': 'stark'}, 1) #das musste ich anpassen für die fusion
df_new = df_new.replace({'donor_alcohol': 'Unbekannt'}, np.NaN)

df_new = df_new.replace({'donor_alt': 0}, np.NaN)
df_new = df_new.replace({'donor_ast': 0}, np.NaN)
df_new = df_new.replace({'donor_ggt': 0}, np.NaN)

df_new = df_new[df_new['recipient_age'] >= 16]
df_new = df_new[df_new['donor_age'] >= 16]

In [ ]:
#Einheiten ausgleichen
df_new.loc[df_new['donor_sodium'] > 180, 'donor_sodium'] = np.NaN
df_new.loc[df_new['donor_sodium'] < 115, 'donor_sodium'] = np.NaN
df_new.loc[df_new['donor_platelets'] > 2000, 'donor_platelets'] = np.NaN
df_new.loc[df_new['donor_bili'] > 100, 'donor_bili'] = np.NaN
df_new.loc[df_new['donor_alt'] > 2000, 'donor_alt'] = np.NaN
df_new.loc[df_new['donor_ast'] > 2000, 'donor_ast'] = np.NaN
df_new.loc[df_new['donor_ggt'] > 2000, 'donor_ggt'] = np.NaN

#BMI Umrechnungen
df_new.loc[df_new['recipient_bmi'] == 'zu viel', 'recipient_bmi'] = np.NaN
df_new.loc[df_new['recipient_bmi'] > 50, 'recipient_bmi'] = np.NaN
df_new.loc[df_new['donor_bmi'] > 100, 'donor_bmi'] = np.NaN

In [33]:
df_new = df_new.convert_dtypes()
print(df_new.dtypes)

year                           Int64
recipient_status               Int64
recipient_age                Float64
recipient_bmi                Float64
recipient_sex                  Int64
recipient_blood_type           Int64
recipient_rhesus_factor        Int64
MELD                           Int64
recipient_followup_days        Int64
recipient_followup_status      Int64
graft_followup_days            Int64
graft_followup_status          Int64
cold_ischemia_time             Int64
donor_age                    Float64
donor_bmi                    Float64
donor_sex                      Int64
donor_blood_type               Int64
donor_rhesus_factor            Int64
donor_region                   Int64
donor_drugs                    Int64
donor_alcohol                  Int64
donor_alt                    Float64
donor_ast                    Float64
donor_ggt                    Float64
donor_bili                   Float64
donor_platelets              Float64
donor_sodium                 Float64
d

In [34]:
percent_missing = df_new.isnull().sum() * 100 / len(df_new)
missing_value_df = pd.DataFrame({'column_name': df_new.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(missing_value_df)

                                         column_name  percent_missing
year                                            year            0.000
donor_rhesus_factor              donor_rhesus_factor            0.000
donor_blood_type                    donor_blood_type            0.000
donor_sex                                  donor_sex            0.000
recipient_rhesus_factor      recipient_rhesus_factor            0.000
donor_age                                  donor_age            0.000
recipient_sex                          recipient_sex            0.000
recipient_blood_type            recipient_blood_type            0.000
recipient_age                          recipient_age            0.000
donor_bmi                                  donor_bmi            0.010
recipient_status                    recipient_status            0.040
recipient_bmi                          recipient_bmi            0.139
MELD                                            MELD            1.983
graft_followup_days 

In [35]:
correlation = df_new['recipient_followup_days'].corr(df_new['graft_followup_days'])
print("Correlation coefficient:", correlation)

Correlation coefficient: 0.9640453906630883


In [36]:
correlation = df_new['recipient_followup_status'].corr(df_new['graft_followup_status'])
print("Correlation coefficient:", correlation)

Correlation coefficient: 0.9426624636810392


In [37]:
df_new['1yr_graft'] = np.NaN
df_new.loc[(df_new["graft_followup_days"] <= 365) & (df_new["graft_followup_status"] ==  1), '1yr_graft'] = 1
df_new.loc[df_new["graft_followup_days"] > 365, '1yr_graft'] = 0

df_new = df_new.dropna(subset=['1yr_graft'])

In [38]:
df_new

,year,recipient_status,recipient_age,recipient_bmi,recipient_sex,recipient_blood_type,recipient_rhesus_factor,MELD,recipient_followup_days,recipient_followup_status,...,donor_region,donor_drugs,donor_alcohol,donor_alt,donor_ast,donor_ggt,donor_bili,donor_platelets,donor_sodium,1yr_graft
0,2013,2,51.643836,25.880571,0,4,1,20,1349,1,...,2,0,0,14.0,24.0,33.0,10.3,218.0,138.0,0.0
1,2014,2,60.057534,21.295295,0,1,0,30,168,1,...,3,<NA>,1,16.0,23.0,35.0,8.0,173.0,151.0,1.0
2,2011,2,71.583562,23.999459,1,3,1,12,414,0,...,6,0,1,35.0,18.0,88.0,13.0,<NA>,143.0,0.0
3,2007,2,65.454795,22.309356,1,4,0,10,2331,0,...,3,0,<NA>,269.0,263.0,63.0,<NA>,177.0,143.0,0.0
4,2006,3,40.323288,18.836565,1,1,0,20,2170,0,...,5,0,0,22.0,63.0,211.0,<NA>,385.0,150.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10359,2019,2,44.986301,19.157088,1,1,0,10,184,1,...,7,0,0,23.0,<NA>,33.0,23.9,181.0,152.0,1.0
10366,2019,2,40.10411,26.621747,1,1,1,26,358,1,...,<NA>,<NA>,<NA>,22.0,37.0,24.0,14.0,165.0,139.0,1.0
10376,2019,2,60.953425,19.723183,1,4,1,30,83,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,144.0,1.0
10381,2019,2,44.967123,26.827421,0,4,1,34,16,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,147.0,1.0


In [1]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(df_new, title="Data Overview", minimal=True)
# profile.to_widgets()
# profile.to_file("reports/ger_1.html")

In [40]:
df_new.to_excel("data/ger_preimp.xlsx")
df_new.to_pickle('data/ger_preimp.p')

# Imputation

In [41]:
import pandas as pd
import pyreadstat
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [42]:
plt.rcParams['figure.figsize'] = (12,8)
sns.heatmap(df_new.notnull(),yticklabels = False, cbar = False, cmap = 'Blues')
plt.title("Missing values")

Text(0.5, 1.0, 'Missing values')

In [43]:
print(df_new.dtypes)

year                           Int64
recipient_status               Int64
recipient_age                Float64
recipient_bmi                Float64
recipient_sex                  Int64
recipient_blood_type           Int64
recipient_rhesus_factor        Int64
MELD                           Int64
recipient_followup_days        Int64
recipient_followup_status      Int64
graft_followup_days            Int64
graft_followup_status          Int64
cold_ischemia_time             Int64
donor_age                    Float64
donor_bmi                    Float64
donor_sex                      Int64
donor_blood_type               Int64
donor_rhesus_factor            Int64
donor_region                   Int64
donor_drugs                    Int64
donor_alcohol                  Int64
donor_alt                    Float64
donor_ast                    Float64
donor_ggt                    Float64
donor_bili                   Float64
donor_platelets              Float64
donor_sodium                 Float64
1

In [44]:
ordinal_cols = [
    "year",
    "recipient_status",
    "recipient_sex",
    "recipient_blood_type",
    "recipient_rhesus_factor",
    "recipient_followup_status",
    "graft_followup_status",
    "donor_sex",
    "donor_blood_type",
    "donor_rhesus_factor",
    "donor_region",
    "donor_drugs",
    "donor_alcohol",
    "1yr_graft",
]

In [45]:
cols_remaining = df_new.columns[df_new.isna().any()].difference(ordinal_cols)
numeric_cols = cols_remaining.tolist()

In [46]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp = IterativeImputer(max_iter=100, random_state=0)
df_imputed = imp.fit_transform(df_new)

In [47]:
df_imputed = pd.DataFrame(df_imputed, columns=df_new.columns, index=df_new.index)

In [48]:
df_imputed[ordinal_cols] = df_imputed[ordinal_cols].round().astype('Int64')

In [49]:
exclude_cols = ['recipient_age', 'recipient_bmi', 'donor_age', 'donor_bmi']
df_imputed.loc[:, df_imputed.columns.difference(exclude_cols)] = (
    df_imputed.loc[:, df_imputed.columns.difference(exclude_cols)].round()
)

In [50]:
df_imputed = df_imputed.where(df_imputed >= 0, 0)

In [51]:
print(df_imputed.dtypes)

year                           Int64
recipient_status               Int64
recipient_age                float64
recipient_bmi                float64
recipient_sex                  Int64
recipient_blood_type           Int64
recipient_rhesus_factor        Int64
MELD                         float64
recipient_followup_days      float64
recipient_followup_status      Int64
graft_followup_days          float64
graft_followup_status          Int64
cold_ischemia_time           float64
donor_age                    float64
donor_bmi                    float64
donor_sex                      Int64
donor_blood_type               Int64
donor_rhesus_factor            Int64
donor_region                   Int64
donor_drugs                    Int64
donor_alcohol                  Int64
donor_alt                    float64
donor_ast                    float64
donor_ggt                    float64
donor_bili                   float64
donor_platelets              float64
donor_sodium                 float64
1

In [52]:
df_imputed

,year,recipient_status,recipient_age,recipient_bmi,recipient_sex,recipient_blood_type,recipient_rhesus_factor,MELD,recipient_followup_days,recipient_followup_status,...,donor_region,donor_drugs,donor_alcohol,donor_alt,donor_ast,donor_ggt,donor_bili,donor_platelets,donor_sodium,1yr_graft
0,2013,2,51.643836,25.880571,0,4,1,20.0,1349.0,1,...,2,0,0,14.0,24.0,33.0,10.0,218.0,138.0,0
1,2014,2,60.057534,21.295295,0,1,0,30.0,168.0,1,...,3,0,1,16.0,23.0,35.0,8.0,173.0,151.0,1
2,2011,2,71.583562,23.999459,1,3,1,12.0,414.0,0,...,6,0,1,35.0,18.0,88.0,13.0,191.0,143.0,0
3,2007,2,65.454795,22.309356,1,4,0,10.0,2331.0,0,...,3,0,0,269.0,263.0,63.0,13.0,177.0,143.0,0
4,2006,3,40.323288,18.836565,1,1,0,20.0,2170.0,0,...,5,0,0,22.0,63.0,211.0,11.0,385.0,150.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10359,2019,2,44.986301,19.157088,1,1,0,10.0,184.0,1,...,7,0,0,23.0,50.0,33.0,24.0,181.0,152.0,1
10366,2019,2,40.104110,26.621747,1,1,1,26.0,358.0,1,...,3,0,0,22.0,37.0,24.0,14.0,165.0,139.0,1
10376,2019,2,60.953425,19.723183,1,4,1,30.0,83.0,1,...,4,0,0,60.0,83.0,74.0,10.0,213.0,144.0,1
10381,2019,2,44.967123,26.827421,0,4,1,34.0,16.0,1,...,4,0,0,53.0,72.0,68.0,9.0,210.0,147.0,1


In [2]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(df_imputed, title="Data Overview After Imputation", minimal=True)
# profile.to_widgets()
# profile.to_file("reports/ger_2.html")

In [54]:
df_imputed.to_excel("data/ger_imp.xlsx")
df_imputed.to_pickle('data/ger_imp.p')

In [55]:
raise Exception("Stop")

Exception: Stop